In [1]:
import itertools
import numpy as np
from qiskit_aer import Aer
from qiskit import QuantumCircuit, QuantumRegister, execute
from qiskit.circuit.library import MCMT, PhaseGate
from qokit.qaoa_objective import get_qaoa_objective

In [2]:
def random_ksat_instance(k, n, m):
    """
    Generate a random k-SAT instance.
    Args:
        k (int): Number of literals per clause.
        n (int): Number of variables.
        m (int): Number of clauses.
    Returns:
        clauses_vars (ndarray): Variables in each clause.
        clauses_negs (ndarray): Negations in each clause.
    """
    clauses_vars = np.zeros((m, k), dtype=int)
    for i in range(m):
        clauses_vars[i] = np.random.choice(n, k, replace=False)
    clauses_negs = np.random.randint(0, 2, size=(m, k))
    return clauses_vars, clauses_negs

def ksat_instance_num_violated(n, clauses_vars, clauses_negs):
    """
    Calculate the number of violated clauses for each assignment.
    """
    bitstrings = (np.arange(2 ** n)[:, None] >> np.arange(n)[None, :]) & 1
    num_violated = np.zeros(2 ** n, dtype=int)
    for (clause_vars, clause_negs) in zip(clauses_vars, clauses_negs):
        num_violated += ~((bitstrings[:, clause_vars] ^ clause_negs).any(axis=1))
    return num_violated

def circuit_probabilities(qc, save_statevector=True, trace_last=False):
    """
    Get probabilities from a quantum circuit.
    """
    backend = Aer.get_backend('statevector_simulator')
    statevector = execute(qc, backend).result().get_statevector()
    probs = np.abs(np.asarray(statevector)) ** 2
    if trace_last:
        probs = probs[:2 ** (len(qc.qubits) - 1)]
    return probs

def circuit_energy(qc, num_violated, m, save_statevector=True, trace_last=False):
    """
    Calculate the energy of a quantum circuit.
    """
    probs = circuit_probabilities(qc, save_statevector=save_statevector, trace_last=trace_last)
    return (probs * (num_violated - m)).sum()

def apply_cost(qc, gamma, terms):
    """
    Apply cost Hamiltonian to the circuit.
    """
    for (term_coeff, term_pauli_z) in terms:
        pauli_z_qubit_index_set = set(term_pauli_z)
        term_pauli_z = tuple(pauli_z_qubit_index_set)
        if len(term_pauli_z) == 0:
            continue
        target = term_pauli_z[-1]
        for control in term_pauli_z[:-1]:
            qc.cx(control, target)
        qc.rz(2 * gamma * term_coeff, target)
        for control in term_pauli_z[:-1]:
            qc.cx(control, target)
        qc.barrier()

def apply_cost_mcmt(qc, gamma, clauses_vars, clauses_negs, k):
    """
    Apply cost using MCMT gates.
    """
    for (clause_vars, clause_negs) in zip(clauses_vars, clauses_negs):
        for ind_negs, negs in enumerate(clause_negs):
            if not negs:
                qc.x(clause_vars[ind_negs])
        qc.append(MCMT(PhaseGate(-gamma), num_ctrl_qubits=k-1, num_target_qubits=1), np.sort(clause_vars).tolist()[::-1])
        for ind_negs, negs in enumerate(clause_negs):
            if not negs:
                qc.x(clause_vars[ind_negs])
        qc.barrier()

def apply_mixer(qc, beta):
    """
    Apply mixer Hamiltonian to the circuit.
    """
    for qubit in qc.qubits:
        qc.rx(2 * beta, qubit)

def get_qaoa_circuit_from_terms(n, terms, betas, gammas):
    """
    Build QAOA circuit from terms.
    """
    qr = QuantumRegister(n)
    qc = QuantumCircuit(qr)
    qc.h(qr)
    for gamma, beta in zip(gammas, betas):
        apply_cost(qc, gamma, terms)
        apply_mixer(qc, beta)
    return qc

def get_qaoa_circuit_mcmt(n, clauses_vars, clauses_negs, betas, gammas, k):
    """
    Build QAOA circuit using MCMT gates.
    """
    qr = QuantumRegister(n)
    qc = QuantumCircuit(qr)
    qc.h(qr)
    for gamma, beta in zip(gammas, betas):
        apply_cost_mcmt(qc, gamma, clauses_vars, clauses_negs, k)
        apply_mixer(qc, beta)
    return qc

In [12]:
np.random.seed(42)

k, r = 4, 9.93
n = 10
m = np.random.poisson(n * r)

clauses_vars, clauses_negs = random_ksat_instance(k, n, m)
num_violated = ksat_instance_num_violated(n, clauses_vars, clauses_negs)
all_satisfied = (num_violated == 0)
satisfying_assignments_as_integers = np.argwhere(num_violated == 0).flatten()
satisfying_assignments_as_bitstrings = (satisfying_assignments_as_integers[:, None] >> np.arange(n)[None, ::-1]) & 1
n_sol = sum(all_satisfied)
assert n_sol > 0
print(f"# satisfying assignments: {n_sol}")

p = 1
qaoa_energy_obj = get_qaoa_objective(
    n,
    p,
    precomputed_diagonal_hamiltonian=num_violated - m,
    objective="expectation",
    parameterization="theta"
)

# Build terms for cost Hamiltonian
terms = []
for (clause_vars, clause_negs) in zip(clauses_vars, clauses_negs):
    for clause_vars_mask in itertools.product([0, 1], repeat=k):
        clause_vars_mask_argwhere = np.argwhere(clause_vars_mask).flatten()
        terms.append(
            (2 ** (-k) * np.prod((-1) ** clause_negs[clause_vars_mask_argwhere]), tuple(clause_vars[clause_vars_mask_argwhere]))
        )

eval_gammas, eval_betas = np.random.rand(p), np.random.rand(p)
qc = get_qaoa_circuit_from_terms(n, terms, eval_betas, eval_gammas)
qc_mcmt = get_qaoa_circuit_mcmt(n, clauses_vars, clauses_negs, eval_betas, eval_gammas, k)

qokit_energy = qaoa_energy_obj(np.concatenate((2 * eval_gammas, eval_betas)))
qiskit_energy = circuit_energy(qc, num_violated, m)
qiskit_mcmt_energy = circuit_energy(qc_mcmt, num_violated, m)

print(f"energy: qokit -> {qokit_energy} | qiskit -> {qiskit_energy} | qiskit_mcmt -> {qiskit_mcmt_energy}")

# satisfying assignments: 2
energy: qokit -> -90.16117625194646 | qiskit -> -90.16117625193662 | qiskit_mcmt -> -90.16117625194647
